Downloads description of articles
Cargo un archivo en forma de dataframe, exitraigo un sub-dataframe, scrapeo el comentario de wallapop y lo retorno en un df que es id - descripcion

In [ ]:
import os
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
# Función de scrapeo comentarios
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import time

In [14]:
# Selecciono el CSV
elemento_a_buscar = "iphone" # para guardar el archivo   <--------
#iphone = int((elemento_a_buscar).split()[1]) # 12, 14 o 15

In [ ]:
# Cargo archivo
fecha = datetime.now().strftime("%Y%m%d")
#fecha = "20250117"
carpeta_origen = "../data/3_feature_engineering"
carpeta_destino = "../data/4_download_description"


archivo = carpeta_origen + '/' + elemento_a_buscar.split()[0] + '_' + fecha + '.xlsx'
print(archivo)
df_original = pd.read_excel(carpeta_origen + '/' + elemento_a_buscar.split()[0] + '_' + fecha + '.xlsx')
df_original.head()


3_feature_engineering/iphone_20250126.xlsx


,id,fecha,gen,mod,memoria,bateria,precio,estado,nombre,scrap_locat,dist_scrap_locat,reservado,url,tiene_color,tiene_emojis,tiene_revisado
0,1033637778,20250126,15,basic,128,NaN,"857,00",good,apple iphone 15 128gb verde - revisado,madrid,60,False,https://es.wallapop.com/item/apple-iphone-15-1...,True,True,True
1,1062144935,20250126,14,pro max,128,NaN,"919,00",as_good_as_new,iphone 14 pro max (128gb) como nuevo,madrid,60,False,https://es.wallapop.com/item/iphone-14-pro-max...,False,True,False
2,1074220444,20250126,15,pro,NaN,NaN,"749,00",as_good_as_new,iphone 15 pro,madrid,60,False,https://es.wallapop.com/item/iphone-15-pro-107...,False,False,False
3,1088399856,20250126,16,basic,512,NaN,"915,00",new,apple iphone 16 512gb nuevo garantia,madrid,60,False,https://es.wallapop.com/item/apple-iphone-16-5...,False,False,False
4,1092712959,20250126,16,pro,128,NaN,"970,00",as_good_as_new,iphone 16 pro / 128gb / titanio natural,madrid,60,False,https://es.wallapop.com/item/iphone-16-pro-128...,True,True,False


In [16]:
# Selecciono el sub-dataframe.
# Selecciono:       as_good_as_new 
#                   iphone 15. 
#                   bateria = NaN
# Son 263 apariciones
# estado = new...
df_to_scrap = df_original[#(df_original['estado'] == estado) & 
                          #(df_original['gen'] == iphone) & 
                          (pd.isna(df_original['bateria']))]
#df_to_scrap = df_original
df_to_scrap.reset_index(drop=True, inplace = True)
df_to_scrap.head(5)

,id,fecha,gen,mod,memoria,bateria,precio,estado,nombre,scrap_locat,dist_scrap_locat,reservado,url,tiene_color,tiene_emojis,tiene_revisado
0,1033637778,20250126,15,basic,128,NaN,"857,00",good,apple iphone 15 128gb verde - revisado,madrid,60,False,https://es.wallapop.com/item/apple-iphone-15-1...,True,True,True
1,1062144935,20250126,14,pro max,128,NaN,"919,00",as_good_as_new,iphone 14 pro max (128gb) como nuevo,madrid,60,False,https://es.wallapop.com/item/iphone-14-pro-max...,False,True,False
2,1074220444,20250126,15,pro,NaN,NaN,"749,00",as_good_as_new,iphone 15 pro,madrid,60,False,https://es.wallapop.com/item/iphone-15-pro-107...,False,False,False
3,1088399856,20250126,16,basic,512,NaN,"915,00",new,apple iphone 16 512gb nuevo garantia,madrid,60,False,https://es.wallapop.com/item/apple-iphone-16-5...,False,False,False
4,1092712959,20250126,16,pro,128,NaN,"970,00",as_good_as_new,iphone 16 pro / 128gb / titanio natural,madrid,60,False,https://es.wallapop.com/item/iphone-16-pro-128...,True,True,False


In [ ]:
# no existe https://es.wallapop.com/item/iphone-15-128gb-como-nuevo-1069439182
# si existe https://es.wallapop.com/item/iphone-15-pro-max-esim-256gb-1069441441

In [18]:

# 263 anuncios - 25 min. con sleep de 1
# 5s por anuncio si va lento. Si va bien 1.5s
# Escrapeo objetos de:
# Vendedor, cuadro datos, cuadro envio, cuadro comentario, cuadro 'editado views y likes'
# por ahora solo comentario


def scrap_comments(df):
    """
    Dado un df con una columna 'url de wallapop':
    accede a la URL, 
    busca el comentario y
    lo guarda en la columna 'comentario'
    """
    df_len = len(df)
    df['comentario'] = None
    if 'url' not in df.columns:
        raise ValueError("Error: La columna 'url' no existe en el DataFrame.")
    print("Tiempo estimado de inicio: 25s...")
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.get(df.loc[0]['url']) 
    WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "onetrust-reject-all-handler"))).click()  # Quita cookies y carga primera web
    for index in range(len(df)):
        start_time = time.time()  # Marca de tiempo al inicio de la iteración
        print(index, '/', df_len)
        driver.get(df.loc[index]['url']) 
        try:
            elemento_comentario = driver.find_element(By.CSS_SELECTOR, "#__next > main > div > div.item-detail_ItemDetail__container__8p25r.pb-5 > section.justify-content-around.d-flex.py-4 > div.item-detail_ItemDetail__card__jnUEv.pb-5 > div.px-4 > div.item-detail_ItemDetail__separator__SCH3p.py-2 > section").text
            df.loc[index, 'comentario'] = elemento_comentario
            print(df.loc[index]['url'], ': \n', elemento_comentario)
        except:
            #element = driver.find_element(By.CLASS_NAME, "not-found-page_Error__title__Dky_8")
            df.loc[index, 'comentario'] = "not found"
            print(df.loc[index]['url'], df.loc[index, 'comentario'])

        end_time = time.time()  # Marca de tiempo al final de la iteración
        execution_time = end_time - start_time  # Calcula el tiempo de ejecución
        print(f"Estimated time to finish: {((df_len - index) * execution_time/60):.2f}min")
        print(f"{execution_time:.2f}s")  
    driver.quit()
    return df

#file = "1_datos_raw/" + "20250102_gopro 13_as_good_as_new.csv"
#df_test = pd.read_csv(file)
df_with_comments = scrap_comments(df_to_scrap)

C:\Users\ACER\AppData\Local\Temp\ipykernel_15808\3237086349.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comentario'] = None


Tiempo estimado de inicio: 25s...
0 / 1019
https://es.wallapop.com/item/apple-iphone-15-128gb-verde-revisado-1033637778 : 
 
Estimated time to finish: 109.35min
6.44s
1 / 1019
https://es.wallapop.com/item/iphone-14-pro-max-128gb-como-nuevo-1062144935 : 
 "Marca: Apple

Modelo: Iphone 14 Pro Max

Capacidad de almacenamiento: 128GB

Color: Plata

Accesorios: incluye cable de carga

🔋 Salud de la batería: +85%

✅ Revisado: funciona correctamente

⭐ LOS MEJOR VALORADOS: 5/5 Estrellas en Google

📦 Envios rápidos a toda España 

✨ El móvil está como nuevo (en apariencia y funcionamiento), impecable! 

🛡 Seguridad y fiabilidad: móvil original de Apple certificado 

🧾 Dispone de factura y 1 año de garantía

📲 (TAMBIÉN DISPONIBLE EN OTROS COLORES Y MEMORIAS)"
Estimated time to finish: 102.26min
6.03s
2 / 1019
https://es.wallapop.com/item/iphone-15-pro-1074220444 : 
 Detalles del producto
iPhone 15 Pro azul titanio de 128 Gb.

Se ha usado durante un año, se vende por cambio a iPhone 16 Pro.

Muy

In [19]:
df_with_comments.head(4)


,id,fecha,gen,mod,memoria,bateria,precio,estado,nombre,scrap_locat,dist_scrap_locat,reservado,url,tiene_color,tiene_emojis,tiene_revisado,comentario
0,1033637778,20250126,15,basic,128,NaN,"857,00",good,apple iphone 15 128gb verde - revisado,madrid,60,False,https://es.wallapop.com/item/apple-iphone-15-1...,True,True,True,
1,1062144935,20250126,14,pro max,128,NaN,"919,00",as_good_as_new,iphone 14 pro max (128gb) como nuevo,madrid,60,False,https://es.wallapop.com/item/iphone-14-pro-max...,False,True,False,"""Marca: Apple\n\nModelo: Iphone 14 Pro Max\n\n..."
2,1074220444,20250126,15,pro,NaN,NaN,"749,00",as_good_as_new,iphone 15 pro,madrid,60,False,https://es.wallapop.com/item/iphone-15-pro-107...,False,False,False,Detalles del producto\niPhone 15 Pro azul tita...
3,1088399856,20250126,16,basic,512,NaN,"915,00",new,apple iphone 16 512gb nuevo garantia,madrid,60,False,https://es.wallapop.com/item/apple-iphone-16-5...,False,False,False,Detalles del producto\n👇👇👇👇👇👇👇👇\n👉NO NEGOCIABL...


In [20]:
subdf_comentarios = df_with_comments[['id', 'comentario']]
subdf_comentarios

,id,comentario
0,1033637778,
1,1062144935,"""Marca: Apple\n\nModelo: Iphone 14 Pro Max\n\n..."
2,1074220444,Detalles del producto\niPhone 15 Pro azul tita...
3,1088399856,Detalles del producto\n👇👇👇👇👇👇👇👇\n👉NO NEGOCIABL...
4,1092712959,Detalles del producto\nSe vende por imprevisto...
...,...,...
1014,1090364053,Apple\nIPHONE 14 PRO 128GB\nBLACK\n\nTeléfono ...
1015,1090364041,Apple\nIPHONE 14 PRO 256GB\nBLACK\n\nTeléfono ...
1016,1090364007,Apple\nIPHONE 14 PRO 128GB\nBLACK\n\nTeléfono ...
1017,1090364010,Apple\nIPHONE 14 PRO 128GB\nBLACK\n\nTeléfono ...


In [21]:
# Guardo comentarios finales en el df original
df_original = pd.merge(df_original, subdf_comentarios, on='id', how='left')


In [22]:
df_original.head(5)

,id,fecha,gen,mod,memoria,bateria,precio,estado,nombre,scrap_locat,dist_scrap_locat,reservado,url,tiene_color,tiene_emojis,tiene_revisado,comentario
0,1033637778,20250126,15,basic,128,NaN,"857,00",good,apple iphone 15 128gb verde - revisado,madrid,60,False,https://es.wallapop.com/item/apple-iphone-15-1...,True,True,True,
1,1062144935,20250126,14,pro max,128,NaN,"919,00",as_good_as_new,iphone 14 pro max (128gb) como nuevo,madrid,60,False,https://es.wallapop.com/item/iphone-14-pro-max...,False,True,False,"""Marca: Apple\n\nModelo: Iphone 14 Pro Max\n\n..."
2,1074220444,20250126,15,pro,NaN,NaN,"749,00",as_good_as_new,iphone 15 pro,madrid,60,False,https://es.wallapop.com/item/iphone-15-pro-107...,False,False,False,Detalles del producto\niPhone 15 Pro azul tita...
3,1088399856,20250126,16,basic,512,NaN,"915,00",new,apple iphone 16 512gb nuevo garantia,madrid,60,False,https://es.wallapop.com/item/apple-iphone-16-5...,False,False,False,Detalles del producto\n👇👇👇👇👇👇👇👇\n👉NO NEGOCIABL...
4,1092712959,20250126,16,pro,128,NaN,"970,00",as_good_as_new,iphone 16 pro / 128gb / titanio natural,madrid,60,False,https://es.wallapop.com/item/iphone-16-pro-128...,True,True,False,Detalles del producto\nSe vende por imprevisto...


In [ ]:
nombre_archivo_csv = df_original
if not os.path.exists(carpeta_destino):
    os.makedirs(carpeta_destino)
nombre_archivo_pkl = fecha + '_' + elemento_a_buscar
df_original.to_csv(carpeta_destino + '/' + nombre_archivo_pkl + '.csv')